In [1]:
import random
import xml.etree.ElementTree as ET

import pandas
import requests
from tqdm import tqdm_notebook

%load_ext rpy2.ipython

In [2]:
tree = ET.parse('../data/tree.xml')
root = tree.getroot()

In [3]:
studies = root.findall('EXPERIMENT_PACKAGE_SET/EXPERIMENT_PACKAGE/STUDY')

study_accessions = [study.get('accession') for study in studies]
# Remove duplicate accessions
study_accessions = list(set(study_accessions))
print(study_accessions[:5])

['SRP030040', 'SRP099599', 'SRP039694', 'SRP022133', 'SRP033464']


In [4]:
%%R
suppressPackageStartupMessages({
    library('recount')
    library('dplyr')
    library('xml2')
    library('biomaRt')
})


# Download the metadata for all samples in recount, and write it to a file
metadata <- all_metadata()
write.table(metadata, '~/Desktop/brd-net/data/recount_metadata.csv', sep='\t', row.names=FALSE)

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2019-07-29 15:18:06 downloading the metadata to /tmp/RtmpBneVGQ/metadata_clean_sra.Rdata

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://github.com/leekgroup/recount-website/blob/master/metadata/metadata_clean_sra.Rdata?raw=true'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Content type 'application/octet-stream'
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2531337 bytes (2.4 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: =
  warnings.warn(x, RRuntimeW

In [5]:
# Load the metadata from calling recount's all_metadata function
recount2_metadata = pandas.read_csv('../data/recount_metadata.csv', sep='\t')

# Create the set of all project accessions present in recount2
recount2_study_accessions = set(recount2_metadata['project'])
print(len(recount2_study_accessions))

3219


In [6]:
# Keep only the accessions that can be downloaded from recount2
study_accessions = [accession for accession in study_accessions if accession in recount2_study_accessions]
print(len(study_accessions))

239


In [7]:
# Download information about a single study to show the data format returned from SRA
data = requests.get('http://metasra.biostat.wisc.edu/api/v01/samples.json?study={}'.format(study_accessions[0]))
print(data.json())

{'sampleCount': 28, 'terms': [{'sampleCount': 28, 'dterm': {'name': 'female organism', 'ids': ['UBERON:0003100']}}, {'sampleCount': 28, 'dterm': {'name': 'liver', 'ids': ['UBERON:0002107']}}, {'sampleCount': 28, 'dterm': {'name': 'male organism', 'ids': ['UBERON:0003101']}}], 'skip': 0, 'studyCount': 1, 'studies': [{'sampleCount': 28, 'study': {'title': 'Homo sapiens Transcriptome or Gene expression', 'id': 'SRP030040', 'recountId': 'SRP030040'}, 'sampleGroups': [{'type': {'type': 'tissue', 'conf': 0.7829097488246092}, 'samples': [{'experiments': [{'runs': ['SRR998141'], 'id': 'SRX355613'}], 'id': 'SRS483808'}], 'dterms': [{'name': 'liver', 'ids': ['UBERON:0002107']}, {'name': 'male organism', 'ids': ['UBERON:0003101']}], 'study': {'title': 'Homo sapiens Transcriptome or Gene expression', 'id': 'SRP030040', 'recountId': 'SRP030040'}, 'attr': [['Sample no', '1'], ['body_site', 'liver'], ['collection_date', '2013'], ['sex', 'M']]}, {'type': {'type': 'tissue', 'conf': 0.8984855186569776},

In [8]:
id_to_studies = {}
id_to_samples = {}
id_to_name = {}

total_samples = 0

# Find the disease ontology information associated with all the studies in both recount2 and 
# the output of find_studies.py
for accession in tqdm_notebook(study_accessions):
    data = requests.get('http://metasra.biostat.wisc.edu/api/v01/samples.json?study={}'.format(accession)).json()
    
    total_samples += data['sampleCount']
    
    for term in data['terms']:
        term_name = term['dterm']['name']
        term_ids = term['dterm']['ids']
        
        # Iterate over all ids in term_ids, and return the disease ontology id.
        # If there is no doid for the term, continue to the next term.
        disease_id = next((id_ for id_ in term_ids if 'DOID' in id_), None)
        if disease_id is None:
            continue
        
        id_to_studies[disease_id] = id_to_studies.setdefault(disease_id, 0) + 1
        id_to_samples[disease_id] = id_to_samples.setdefault(disease_id, 0) + term['sampleCount']
        id_to_name[disease_id] = term_name

In [9]:
disease_dict = {}
for id_ in id_to_studies:
    disease_dict[id_] = [id_to_studies[id_], id_to_samples[id_], id_to_name[id_]]

disease_df = pandas.DataFrame.from_dict(disease_dict, orient='index', columns=['num_studies', 'num_samples', 'name'])

all_doids = set(id_to_studies.keys())

disease_df.sort_values('num_samples').head()

,num_studies,num_samples,name
DOID:11722,1,6,myotonic dystrophy type 1
DOID:1974,1,6,adenosarcoma
DOID:12849,1,6,autistic disorder
DOID:0050749,1,8,peripheral T-cell lymphoma
DOID:3965,1,8,Merkel cell carcinoma


In [10]:
DOID_to_descendants = {}
def get_descendants(doid):
    '''This function calls the Disease Ontology API to find the ids of all descendants of a given doid'''
    if doid in DOID_to_descendants:
        return DOID_to_descendants[doid]
    
    descendants = []
    data = requests.get('http://www.disease-ontology.org/api/metadata/{}'.format(doid)).json()
    
    if 'children' not in data:
        return []
    
    for child in data['children']:
        child_doid = child[1]
        descendants.append(child_doid)
        
        # recursively traverse the ontology
        child_descendants = get_descendants(child_doid)
        descendants.extend(child_descendants)
        
    DOID_to_descendants[doid] = descendants
    return descendants

In [11]:
# Iterate through all terms found in the data, and only add them to a list if 
# none of their descendants are in the study

leaf_ids = []

# Because this function involves recursively traversing a tree, the estimated time
# given by tqdm will be way off. That said, it takes awhile to run (~ 20 minutes)

for doid in tqdm_notebook(id_to_name):
    descendants = get_descendants(doid)
    descendant_in_study = False
    for descendant in descendants:
        if descendant in all_doids:
            descendant_in_study = True
            break
    
    if not descendant_in_study:
        leaf_ids.append(doid)

# These aren't necessarily leaves of the ontology, but they are don't have any descendants
# in this dataset, so we'll call them leaves
print(len(leaf_ids))


102


In [12]:
disease_df.loc[leaf_ids].head()

,num_studies,num_samples,name
DOID:684,8,275,hepatocellular carcinoma
DOID:332,4,260,amyotrophic lateral sclerosis
DOID:3965,1,8,Merkel cell carcinoma
DOID:2513,2,29,basal cell carcinoma
DOID:1909,8,239,melanoma


In [13]:
print(total_samples)
print(disease_df.loc[leaf_ids]['num_samples'].sum())

13437
8877


In [14]:
# Select studies at random from the list of all samples
# If the study has one of the leaf ids, add it to the list
# Stop once enough samples are included (~1000 valid samples)

# In a script, you would put the random seed at the top. Since each cell can be
# rerun in a notebook, manually setting the seed within each cell with a random
# call ensures that you get the same results regardless of the order in which
# the cells are run
random.seed(42)

leaf_set = set(leaf_ids)

samples_drawn = 0
plier_sample_accessions = []

# Select samples in a random order
indices = list(range(len(study_accessions)))
random.shuffle(indices)

for index in tqdm_notebook(indices):
    study = study_accessions[index]
    data = requests.get('http://metasra.biostat.wisc.edu/api/v01/samples.json?study={}'.format(study)).json()

    for term in data['terms']:
        term_ids = term['dterm']['ids']

        use_sample = False
        for id_ in term_ids:
            if id_ in leaf_set:
                plier_sample_accessions.append(study)
                
                samples_drawn += data['sampleCount']
                use_sample = True
                break
        
        if use_sample:
            break
            
    # Some of the samples will turn out to be invalid, so we use 3000 here instead of
    # a proportion of the total number of samples
    if samples_drawn > 3000:
        break
        
print(plier_sample_accessions)

['SRP009251', 'SRP051825', 'SRP056041', 'SRP047192', 'SRP042184', 'SRP007947', 'SRP042228', 'SRP002272', 'SRP055874', 'SRP065812', 'SRP014675', 'SRP041538', 'SRP035524', 'SRP043085', 'SRP063838', 'SRP031459', 'SRP002628', 'SRP029880', 'SRP052896', 'SRP034698', 'ERP001908', 'SRP033464', 'SRP057118', 'SRP007461', 'SRP002326', 'SRP037982', 'SRP055438', 'SRP012656', 'ERP009437', 'SRP056792', 'SRP007825', 'SRP050493', 'SRP029434', 'SRP061240', 'SRP059172', 'SRP007483', 'SRP058626', 'SRP007946', 'SRP059039', 'SRP015668', 'SRP058667', 'SRP043391', 'ERP004592', 'SRP011924', 'SRP049820', 'SRP017262', 'SRP056293', 'SRP051765', 'SRP063661', 'SRP041736', 'ERP010889', 'SRP061888', 'SRP041531', 'SRP023262', 'SRP006575']


In [15]:
# Get all runs from xml
experiment_packages = root.findall('EXPERIMENT_PACKAGE_SET/EXPERIMENT_PACKAGE')

plier_healthy = []
plier_disease = []
classifier_healthy = []
classifier_disease = []

recount_study_set = set(study_accessions)

# Find the run accessions and sort them into groups
for experiment_package in experiment_packages:
    study_accession = experiment_package.find('STUDY').get('accession')
    in_plier_set = study_accession in plier_sample_accessions
    
    case_control_status = experiment_package.get('category')
    
    if case_control_status == 'invalid' or study_accession not in recount_study_set:
        continue
    
    
    runs = experiment_package.findall('RUN_SET/RUN/IDENTIFIERS/PRIMARY_ID')
    for run in runs:
        run_id = '.'.join([study_accession, run.text])
        
        if case_control_status == 'control':
            if in_plier_set:
                plier_healthy.append(run_id)
            else:
                classifier_healthy.append(run_id)
        elif case_control_status == 'case':
            if in_plier_set:
                plier_disease.append(run_id)
            else:
                classifier_disease.append(run_id)
      

In [16]:
print(len(plier_healthy))
print(len(plier_disease))
    
print(len(classifier_healthy))
print(len(classifier_disease))

print(plier_healthy[:5])

363
977
3284
3875
['SRP061240.SRR2105249', 'SRP061240.SRR2105250', 'SRP061240.SRR2105243', 'SRP061240.SRR2105244', 'SRP061240.SRR2105221']


## Downloading Data
---
We'll now switch to R, because the only programmatic way to access recount2 data is through the recount bioconductor package. Thanks to the magic of rpy, we can use the list of run accessions we generated in the R code below.

The R portion of this notebook is based heavily on Qiewen Hu's script [here](https://github.com/greenelab/rheum-plier-data/blob/master/recount2/1-get_all_recount_dataset.R)

In [17]:
%%R 
`%>%` <- dplyr::`%>%`

In [18]:
%%R
# Get RPKM value for each gene - adapted from recount package
getRPKM <- function(rse, length_var = 'bp_length', mapped_var = NULL) { 
  # Computes the RPKM value for each gene in the sample.
  #
  # Args: 
  #  rse: A RangedSummarizedExperiment-class object in recount package
  #  length_var: A length 1 character vector with the column name from rowData(rse) that has
  #              the coding length. For gene level objects from recount this is bp_length. If
  #              NULL, then it will use width(rowRanges(rse)) which should be used for exon RSEs.
  #  mapped_var: A length 1 character vector with the column name from colData(rse) that has
  #              the number of reads mapped. If NULL (default) then it will use the column 
  #              sums of the counts matrix
  # Returns:
  #   RPKM value for each sample
  if(!is.null(mapped_var)){
    mapped <- colData(rse)[, mapped_var] 
  } else {
    mapped <- colSums(assays(rse)$counts) 
  } 
  bg <- matrix(mapped, ncol = ncol(rse), nrow = nrow(rse), byrow = TRUE) 
  if(!is.null(length_var)){
    len <- rowData(rse)[, length_var] 
  } else {
    len <- width(rowRanges(rse))
  }
  wid <- matrix(len, nrow = nrow(rse), ncol = ncol(rse), byrow = FALSE) 
  rpkm <- assays(rse)$counts / (wid/1000) / (bg/1e6) 
  return(rpkm)
} 

getExperimentDf <- function(rpkm.list, id.vector) {
  # Extracts the runs found in id.vector from a list of dataframes, and combines
  # them into a single large dataframe
    
  rpkm.df <- do.call(base::cbind, c(rpkm.list, by = "id"))
  subset.df <- rpkm.df %>% dplyr::select(dplyr::one_of(id.vector))
  subset.df <- tibble::rownames_to_column(subset.df, "ENSG")
  
  return(subset.df)
}

In [19]:
%%R

data.dir <- file.path("../data")
recount.dir <- file.path(data.dir, "recount")
dir.create(recount.dir, recursive = TRUE, showWarnings = FALSE)

In [20]:
%%R -i study_accessions

included.study.list <- study_accessions

# Download all recount2 samples in included.study.list that are available
# This takes a while
suppressMessages(
    lapply(included.study.list, 
           function(x) tryCatch(download_study(x, type = "rse-gene", 
                                               outdir = file.path(recount.dir, x)),
                                error= function(e) NULL))
)

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP030040/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Content type 'application/octet-stream'
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 5235067 bytes (5.0 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: =
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 11367458 bytes (10.8 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 10.8 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP025982/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 199976024 bytes (190.7 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 190.7 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 5171682 bytes (4.9 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 4.9 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP048801/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 36435039 bytes (34.7 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 34.7 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:14

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 3900901 bytes (3.7 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP007825/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 4931023 bytes (4.7 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP020486/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2116131 bytes (2.0 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brd

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 5666557 bytes (5.4 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 5.4 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP050971/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 7628683 bytes (7.3 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 7.3 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: 

...

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP052056/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 4817373 bytes (4.6 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP064316/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2336254 bytes (2.2 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP058036/rse_gene.Rdata'

  warnings.warn(x, RRun

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP026562/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2186395 bytes (2.1 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/ERP002414/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 1762440 bytes (1.7 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP014146/rse_gene.Rdata'

  warnings.warn(x, RRun

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP001371/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 24226737 bytes (23.1 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 23.1 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP059172/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 17637895 bytes (16.8 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 21.2 MB


  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP014142/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 2210222 bytes (2.1 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'http://duffel.rail.bio/recount/v2/SRP053296/rse_gene.Rdata'

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 4300274 bytes (4.1 MB)

  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/pyt

[[1]]
[1] "http://duffel.rail.bio/recount/v2/SRP030040/rse_gene.Rdata"

[[2]]
[1] "http://duffel.rail.bio/recount/v2/SRP039694/rse_gene.Rdata"

[[3]]
[1] "http://duffel.rail.bio/recount/v2/SRP022133/rse_gene.Rdata"

[[4]]
[1] "http://duffel.rail.bio/recount/v2/SRP033464/rse_gene.Rdata"

[[5]]
[1] "http://duffel.rail.bio/recount/v2/SRP007338/rse_gene.Rdata"

[[6]]
[1] "http://duffel.rail.bio/recount/v2/SRP034698/rse_gene.Rdata"

[[7]]
[1] "http://duffel.rail.bio/recount/v2/ERP006650/rse_gene.Rdata"

[[8]]
[1] "http://duffel.rail.bio/recount/v2/SRP009123/rse_gene.Rdata"

[[9]]
[1] "http://duffel.rail.bio/recount/v2/SRP033566/rse_gene.Rdata"

[[10]]
[1] "http://duffel.rail.bio/recount/v2/SRP059989/rse_gene.Rdata"

[[11]]
[1] "http://duffel.rail.bio/recount/v2/SRP059170/rse_gene.Rdata"

[[12]]
[1] "http://duffel.rail.bio/recount/v2/DRP000987/rse_gene.Rdata"

[[13]]
[1] "http://duffel.rail.bio/recount/v2/SRP043085/rse_gene.Rdata"

[[14]]
[1] "http://duffel.rail.bio/recount/v2/SRP055440/rse_

In [21]:
%%R

# get RPKM for each experiment and add to list
rpkm.list <- list()
for(experiment in included.study.list) {
  possibleError <- tryCatch(load(file.path(recount.dir, experiment, 'rse_gene.Rdata')), error=function(e) e)
  # If the file can't be loaded, it wasn't in recount, so we don't want to try to add it to our
  # gene expression matrix
  if (inherits(possibleError, 'error')){
    next
  }
  
  rpkm <- as.data.frame(getRPKM(rse_gene))
  rpkm$id <- rownames(rpkm)
  rpkm.list[[experiment]] <- rpkm
}

In [22]:
%%R -i plier_healthy,plier_disease,classifier_healthy,classifier_disease


# Convert our lists of strings to vectors of strings, and change their names to match
# R naming conventions
plier.healthy <- unlist(plier_healthy)
plier.disease <- unlist(plier_disease)
classifier.healthy <- unlist(classifier_healthy)
classifier.disease <- unlist(classifier_disease)

plier.healthy.df <- getExperimentDf(rpkm.list, plier.healthy)
plier.disease.df <- getExperimentDf(rpkm.list, plier.disease)
classifier.healthy.df <- getExperimentDf(rpkm.list, classifier.healthy)
classifier.disease.df <- getExperimentDf(rpkm.list, classifier.disease)

print(dim(plier.healthy.df))
print(dim(plier.disease.df))
print(dim(classifier.healthy.df))
print(dim(classifier.disease.df))

[1] 58037   352
[1] 58037   951
[1] 58037  3156
[1] 58037  2274


In [23]:
%%R 

removeParY <- function (expression.df){
    # Remove PAR_Y duplicate gene versions
    return (anti_join(expression.df, subset(expression.df, grepl('PAR_Y', expression.df['ENSG'][[1]]))))
}

mart <- useDataset('hsapiens_gene_ensembl', mart=useMart('ensembl'))

plier.healthy.df <- removeParY(plier.healthy.df)
plier.disease.df <- removeParY(plier.disease.df)
classifier.healthy.df <- removeParY(classifier.healthy.df)
classifier.disease.df <- removeParY(classifier.disease.df)

# All four subsets of the data contain the same genes, so any could be used here
raw.ids <- plier.healthy.df['ENSG']
ids <- lapply(raw.ids, sub, pattern='\\..*', replacement='')

plier.healthy.df['idprefix'] <- ids
plier.disease.df['idprefix'] <- ids
classifier.healthy.df['idprefix'] <- ids
classifier.disease.df['idprefix'] <- ids


# Query biomaRt to get the Ensembl ID to gene symbol mapping
gene.names <- getBM(filters='ensembl_gene_id', attributes = c('ensembl_gene_id', 'hgnc_symbol'), 
                    values=unlist(ids), mart=mart, uniqueRows = TRUE)
    

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Joining, by = c("ENSG", "SRP061240.SRR2105249", "SRP061240.SRR2105250", "SRP061240.SRR2105243", "SRP061240.SRR2105244", "SRP061240.SRR2105221", "SRP061240.SRR2105222", "SRP061240.SRR2105159", "SRP061240.SRR2105160", "SRP061240.SRR2105167", "SRP061240.SRR2105168", "SRP061240.SRR2105499", "SRP061240.SRR2105500", "SRP061240.SRR2105207", "SRP061240.SRR2105208", "SRP061240.SRR2105173", "SRP061240.SRR2105174", "SRP061240.SRR2105161", "SRP061240.SRR2105162", "SRP061240.SRR2105169", "SRP061240.SRR2105170", "SRP061240.SRR2105253", "SRP061240.SRR2105254", "SRP061240.SRR2105251", "SRP061240.SRR2105252", "SRP061240.SRR2105211", "SRP061240.SRR2105212", "SRP061240.SRR2105163", "SRP061240.SRR2105164", "SRP061240.SRR2105209", "SRP061240.SRR2105210", "SRP061240.SRR2105171", "SRP061240.SRR2105172", "SRP061240.SRR2105165", "SRP061240.SRR2105166", "SRP061240.SRR2105245", "SRP061240.SRR2105246", "S

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Joining, by = c("ENSG", "SRP006574.SRR191462", "SRP006574.SRR191503", "SRP006574.SRR191441", "SRP006574.SRR191442", "SRP006574.SRR191523", "SRP006574.SRR191443", "SRP006574.SRR191444", "SRP006574.SRR191483", "SRP006574.SRR191445", "SRP006574.SRR191463", "SRP006574.SRR191464", "SRP006574.SRR191465", "SRP006574.SRR191466", "SRP006574.SRR191467", "SRP006574.SRR191468", "SRP006574.SRR191469", "SRP006574.SRR191470", "SRP006574.SRR191471", "SRP006574.SRR191524", "SRP006574.SRR191484", "SRP006574.SRR191472", "SRP006574.SRR191473", "SRP006574.SRR191525", "SRP006574.SRR191420", "SRP006574.SRR191421", "SRP006574.SRR191485", "SRP006574.SRR191543", "SRP006574.SRR191504", "SRP006574.SRR191486", "SRP006574.SRR191487", "SRP006574.SRR191544", "SRP006574.SRR191526", "SRP006574.SRR191527", "SRP006574.SRR191422", "SRP006574.SRR191528", "SRP006574.SRR191529", "SRP006574.SRR191530", "SRP006574.SRR1

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [>------------------------------]   3% eta: 27s
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [>------------------------------]   3% eta: 28s
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [>------------------------------]   4% eta: 29s
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [=>-----------------------------]   5% eta: 35s
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [===========>-------------------]  39% eta:  1m
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [===========>-------------------]  40% eta:  1m
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [============>------------------]  41% eta:  1m
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [============>------------------]  42% eta:  1m
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [=====================>---------]  72% eta: 26s
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [======================>--------]  73% eta: 25s
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [======================>--------]  74% eta: 24s
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query [======================>--------]  75% eta: 23s
  warnings.warn(x, RRuntimeWarning)
/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Batch submitting query

In [24]:
%%R
duplicate.symbols <- subset(gene.names, duplicated(gene.names['hgnc_symbol']))
gene.names <- anti_join(gene.names, duplicate.symbols)
print(dim(gene.names))
print(dim(duplicate.symbols))
print(dim(gene.names))
# Some ENSG ids don't have gene symbols. We'll remove them as PLIER gene sets use gene symbols 
gene.names['hgnc_symbol'] <- na_if(gene.names['hgnc_symbol'], '')
gene.names <- na.omit(gene.names)
print(dim(gene.names))

/home/ben/anaconda3/envs/brdnet/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Joining, by = c("ensembl_gene_id", "hgnc_symbol")

  warnings.warn(x, RRuntimeWarning)


[1] 37126     2
[1] 20084     2
[1] 37126     2
[1] 37125     2


In [25]:
%%R
prepare.df <- function(expression.df, gene.names) {
    # prepare.df removes extraneous columns, sets the row names to gene symbols,
    # and generally prepares a dataframe for use in PLIER
    
    merged.df <- merge(expression.df, gene.names, by.x = 'idprefix', by.y = 'ensembl_gene_id')
    
    print(dim(merged.df))

    row.names(merged.df) <- merged.df['hgnc_symbol'][[1]]

    # PLIER panics if there are strings in the data, so remove the id fields
    no.id.df <- merged.df
    no.id.df['ENSG'] <- NULL
    no.id.df['idprefix'] <- NULL
    no.id.df['hgnc_symbol'] <- NULL
    
    return (no.id.df)
}

plier.healthy.df <- prepare.df(plier.healthy.df, gene.names)
plier.disease.df <- prepare.df(plier.disease.df, gene.names)
classifier.healthy.df <- prepare.df(classifier.healthy.df, gene.names)
classifier.disease.df <- prepare.df(classifier.disease.df, gene.names)

[1] 37125   354
[1] 37125   953
[1] 37125  3158
[1] 37125  2276


In [ ]:
%%R

write.table(plier.healthy.df, file.path(data.dir, 'plier_healthy.tsv'), sep='\t', row.names=TRUE)
write.table(plier.disease.df, file.path(data.dir, 'plier_disease.tsv'), sep='\t', row.names=TRUE)
write.table(classifier.healthy.df, file.path(data.dir, 'classifier_healthy.tsv'), sep='\t', row.names=TRUE)
write.table(classifier.disease.df, file.path(data.dir, 'classifier_disease.tsv'), sep='\t', row.names=TRUE)